## Exemple du fonctionnement de l'algorithme de chiffrement utilisé par SEAL
* SEAL est une librairie de chiffrement homomorphe par Microsoft

https://eprint.iacr.org/2017/224.pdf

* SEAL est basé sur le chiffrement BFV

https://eprint.iacr.org/2012/144.pdf

* Code inspiré par

https://arxiv.org/pdf/1906.07127.pdf

* Implémenté sur le framework Python SageMath

https://www.sagemath.org

In [1]:
from sage.stats.distributions.discrete_gaussian_integer import DiscreteGaussianDistributionIntegerSampler

In [2]:
# Modulus in the ciphertext space
q = 2^35 - 2^14 + 2^11 + 1
# Modulus in the plaintext space (doit etre au moins 2x plus grand que le nombre a representer)
t = 128
# A power of 2
n = 1024
# The polynomial modulus which specifies the ring R
f = x^n + 1
# The ring Z[x]/f(x)
P.<x> = PolynomialRing(ZZ)
R.<x> = QuotientRing(P, P.ideal(f))
# Error distribution (a truncated discrete Gaussian distribution)
noise_standard_deviation = 3.20
dist = DiscreteGaussianDistributionIntegerSampler(sigma=noise_standard_deviation)
# Quotient on division of q by t
delta = floor(q / t)

In [3]:
def rq(poly):
    coefs = poly.list()
    for i in range(len(coefs)):
        coefs[i] = coefs[i] % q
    return R(coefs)

def rt(poly):
    coefs = poly.list()
    for i in range(len(coefs)):
        coefs[i] = coefs[i] % t
    return R(coefs)

# Cyphertext Space
def Roundq(poly):
    coefs = poly.list()
    for i in range(len(coefs)):
        coefs[i] = coefs[i] % q
        if coefs[i] > q / 2:
            coefs[i] = coefs[i] - q
    return R(coefs)

# Plaintext Space 
def Roundt(poly):
    coefs = poly.list()
    for i in range(len(coefs)):
        coefs[i] = coefs[i] % t
        if coefs[i] > t / 2:
            coefs[i] = coefs[i] - t
    return R(coefs)

In [4]:
assert q % (2*n) == 1
assert delta * t + rt(q) == q

In [5]:
# Generate a random ternary polynomial (coefs are either -1, 0 or 1)
def sample_poly_ternary():
    return R([ZZ.random_element(-1, 2, "uniform") for _ in range(n)])

# Generate a random polynomial with integer coefs from a Gaussian Distribution χ
def sample_poly_normal():
    return R([dist() for _ in range(n)])
    
# Generate a random polynomial with coefs in Z/qZ
def sample_Rq():
    return R([ZZ.random_element(0, q - 1, "uniform") for _ in range(n)])

## Génération de clé
* Dans SEAL, la clé privé est un polynome cyclotomique

In [6]:
# Generate a secret key
def secretKeyGen():
    return sample_poly_ternary()

### Calcule de la clé publique correspondante
$$a \leftarrow R_q, e \leftarrow \chi$$
$$pk = \big(-[a*s+e]_q, a\big)$$

In [7]:
def publicKeyGen(s):
    a = sample_Rq()
    e = sample_poly_normal()
    return [-Roundq(a * s + e), a]

In [8]:
sk = secretKeyGen()
pk = publicKeyGen(sk)

## La fonction de chiffrement

In [9]:
def encrypt(p, m):
    u = sample_poly_ternary()
    e1 = sample_poly_normal()
    e2 = sample_poly_normal()
    return [Roundq(p[0] * u + e1 + delta * m), Roundq(p[1] * u + e2)]

def decrypt(s, c):
    tmp = t * Roundq(c[0] + c[1] * s)
    coefs = tmp.list()
    for i in range(len(coefs)):
        coefs[i] = round(coefs[i] / q)
    return Roundt(R(coefs))

In [10]:
assert decrypt(sk, encrypt(pk, 1)) == 1

## Addition
$$\Big(\big[c_1[0] + c_2[0]\big]_q , \big[c_1[1] + c_2[1]\big]_q \Big)$$

In [11]:
def add(c1, c2):
    return [Roundq(c1[0] + c2[0]), Roundq(c1[1] + c2[1])]

### Test
Ajoutons 20 à 22

In [12]:
c1 = encrypt(pk, 20)
c2 = encrypt(pk, 22)
print(decrypt(sk, add(c1, c2)))

42


## TODO: la multiplication
L'algorithme de multiplication de BFV est beaucoup plus compliqué que l'addition.
Il se fait en plusieurs étapes et nécessite des changements de bases.

Voir chapitre 4 de [Somewhat Practical Fully Homomorphic Encryption](https://eprint.iacr.org/2012/144.pdf) 

### Exemple d'une clé privée

In [13]:
print(sk)

x^1019 + x^1016 - x^1014 - x^1013 - x^1012 + x^1011 - x^1009 + x^1008 - x^1007 - x^1006 - x^1005 + x^1004 + x^1003 - x^1002 - x^1000 - x^999 + x^998 + x^995 - x^994 + x^992 - x^991 + x^990 - x^989 + x^988 - x^986 + x^985 + x^982 - x^980 - x^978 + x^977 + x^975 - x^974 - x^972 + x^971 + x^969 - x^968 + x^967 + x^966 - x^964 - x^963 - x^962 - x^961 + x^960 - x^959 - x^958 - x^955 + x^953 - x^952 + x^950 - x^947 + x^945 - x^943 - x^942 - x^938 + x^937 + x^936 + x^935 + x^934 - x^933 - x^929 + x^928 + x^926 + x^925 + x^923 + x^922 - x^919 - x^918 - x^917 - x^916 - x^915 - x^914 - x^913 + x^912 - x^911 + x^908 - x^907 - x^906 - x^905 + x^904 + x^903 + x^902 - x^901 - x^900 - x^898 + x^897 - x^896 + x^895 + x^894 - x^892 + x^890 - x^889 - x^888 - x^887 + x^886 - x^883 + x^881 + x^880 + x^879 + x^878 + x^877 - x^876 - x^875 + x^873 - x^872 - x^871 - x^869 + x^868 + x^867 + x^866 + x^865 - x^864 + x^863 - x^862 + x^861 - x^860 - x^859 - x^858 - x^857 + x^856 - x^855 + x^853 + x^851 + x^850 + x

### Exemple d'un nombre chiffré

In [14]:
print(c1)

[3932394031*x^1023 + 13850043939*x^1022 + 15333543911*x^1021 - 11418990717*x^1020 - 282377970*x^1019 - 2008384493*x^1018 - 11037949409*x^1017 - 8515658308*x^1016 + 13993320670*x^1015 - 13932354111*x^1014 + 3507859338*x^1013 + 1162945935*x^1012 + 5579883608*x^1011 + 12832073749*x^1010 + 3026975384*x^1009 - 8747649814*x^1008 - 5879871867*x^1007 - 11510458202*x^1006 + 383019832*x^1005 - 11918237527*x^1004 + 13060102000*x^1003 + 7817384427*x^1002 - 13628019149*x^1001 - 423123457*x^1000 - 7075655851*x^999 - 11885889916*x^998 + 3980561357*x^997 - 808849558*x^996 + 10680856227*x^995 - 12141212752*x^994 - 1940702843*x^993 - 13638291921*x^992 + 1341633742*x^991 - 17093256573*x^990 + 5719913575*x^989 - 16567704237*x^988 - 16607553417*x^987 + 16127937376*x^986 - 15128570970*x^985 + 14706026370*x^984 - 2856590854*x^983 - 15146164843*x^982 + 12327481570*x^981 - 292515343*x^980 + 12635373688*x^979 + 12873004762*x^978 - 12393350311*x^977 - 14632215405*x^976 - 5583237572*x^975 + 10241471567*x^974 + 62